In [1]:
import numpy as np
import pandas as pd
import SimpleITK as sitk
from tqdm import tqdm
import scipy.ndimage
import cv2
import os
import sys
def display_full(x):
    with pd.option_context("display.max_rows", None,
                           "display.max_columns", None,
                           "display.width", 20000,
                           "display.max_colwidth", None,
                           ):
        print(x)

In [2]:
selected_exams = pd.read_excel("/media/andres/T7 Shield1/UCAN_project/Finalized_dataset_1805_exams_with_Age.xlsx")
collages_df = pd.read_excel("/media/andres/T7 Shield1/UCAN_project/df_for_collages.xlsx")
reshaped_projections_path = "/media/andres/T7 Shield1/UCAN_project/2D_projections/reshaped_projections/"

In [3]:
patiend_ID_num_slices_df = selected_exams[["patient_id", "num_slices"]]
patiend_ID_num_slices_df = patiend_ID_num_slices_df.rename(columns={"patient_id": "patient_ID"})
display_full(patiend_ID_num_slices_df.head())

        patient_ID  num_slices
0  lpr385705046400         215
1  lpr385705046400         247
2  lpr415675513429         382
3  lpr415675513429         341
4  lpr415675513429        1337


In [4]:
collages_df.head()

,Unnamed: 0,patient_ID,scan_date,SUV_MIP,CT_MIP,SUV_bone,CT_bone,SUV_lean,CT_lean,SUV_adipose,CT_adipose,SUV_air,CT_air,angle
0,0,lpr385705046400,20140313,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,-90
1,1,lpr385705046400,20140313,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,0
2,2,lpr415675513429,20190201,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,-90
3,3,lpr415675513429,20190201,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,0
4,4,lpr415675513429,20190320,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,-90


In [5]:
collages_df["dimensions"] = 0

In [6]:
for row, image in collages_df.iterrows():
    temp_array = np.load(image["SUV_MIP"])
    collages_df.loc[row, "dimensions"] = temp_array.shape[0]

display_full(collages_df["dimensions"].head(25))

0     362
1     362
2     501
3     501
4     421
5     421
6     501
7     501
8     421
9     421
10    421
11    421
12    362
13    362
14    303
15    303
16    501
17    501
18    461
19    461
20    421
21    421
22    362
23    362
24    421
Name: dimensions, dtype: int64


In [7]:
print(f"Highest number of dimensions: {collages_df['dimensions'].max()}")
print(np.unique(collages_df["dimensions"], return_counts=True))

Highest number of dimensions: 1056
(array([ 303,  329,  342,  362,  374,  395,  412,  421,  445,  448,  461,
        480,  501,  507,  527,  539,  580,  594,  660,  726,  740,  844,
        924, 1003, 1056]), array([  40,    2,    2,  996,    6,  132,    2, 1080,    2,    6,  540,
         46,  606,   10,   88,    2,   56,    6,    6,    4,    2,    2,
          4,    4,    2]))


In [8]:
cropped_array = np.zeros((580, 256))

for row, image in collages_df.iterrows():

    save_path_temp = os.path.join(reshaped_projections_path, str(image["patient_ID"]), str(image["scan_date"]))
    if not os.path.exists(save_path_temp):
        os.makedirs(save_path_temp)

    SUV_MIP = np.load(image["SUV_MIP"])
    size = SUV_MIP.shape[0]

    if size <= 580:
        temp_pad_int = (580 - size)//2
        temp_pad_float = (580 - size)/2
        pad_from_top = temp_pad_int
        pad_from_bottom = temp_pad_int

        if temp_pad_int < temp_pad_float:
            pad_from_top = temp_pad_int + 1

        result = np.pad(SUV_MIP, ((pad_from_top, pad_from_bottom), (0,0)))
        np.save(os.path.join(save_path_temp, "SUV_MIP" + ".npy"), result)

        SUV_bone = np.load(image["SUV_bone"])
        result = np.pad(SUV_MIP, ((pad_from_top, pad_from_bottom), (0,0)))
        np.save(os.path.join(save_path_temp, "SUV_bone" + ".npy"), result)

        SUV_lean = np.load(image["SUV_lean"])
        result = np.pad(SUV_MIP, ((pad_from_top, pad_from_bottom), (0,0)))
        np.save(os.path.join(save_path_temp, "SUV_lean" + ".npy"), result)

        SUV_adipose = np.load(image["SUV_adipose"])
        result = np.pad(SUV_MIP, ((pad_from_top, pad_from_bottom), (0,0)))
        np.save(os.path.join(save_path_temp, "SUV_adipose" + ".npy"), result)

        SUV_air = np.load(image["SUV_air"])
        result = np.pad(SUV_MIP, ((pad_from_top, pad_from_bottom), (0,0)))
        np.save(os.path.join(save_path_temp, "SUV_air" + ".npy"), result)
        
        CT_MIP = np.load(image["CT_MIP"])
        result = np.pad(SUV_MIP, ((pad_from_top, pad_from_bottom), (0,0)))
        np.save(os.path.join(save_path_temp, "CT_MIP" + ".npy"), result)

        CT_bone = np.load(image["CT_bone"])
        result = np.pad(SUV_MIP, ((pad_from_top, pad_from_bottom), (0,0)))
        np.save(os.path.join(save_path_temp, "CT_bone" + ".npy"), result)

        CT_lean = np.load(image["CT_lean"])
        result = np.pad(SUV_MIP, ((pad_from_top, pad_from_bottom), (0,0)))
        np.save(os.path.join(save_path_temp, "CT_lean" + ".npy"), result)

        CT_adipose = np.load(image["CT_adipose"])
        result = np.pad(SUV_MIP, ((pad_from_top, pad_from_bottom), (0,0)))
        np.save(os.path.join(save_path_temp, "CT_adipose" + ".npy"), result)

        CT_air = np.load(image["CT_air"])
        result = np.pad(SUV_MIP, ((pad_from_top, pad_from_bottom), (0,0)))
        np.save(os.path.join(save_path_temp, "CT_air" + ".npy"), result)

    else:
        temp_crop_int = (size - 580)//2
        temp_crop_float = (size - 580)/2
        crop_from_top = temp_crop_int
        crop_from_bottom = crop_from_top + 580

        if temp_crop_int < temp_crop_float:
            crop_from_top = temp_crop_int + 1

        cropped_array = SUV_MIP[crop_from_top:crop_from_bottom, :]
        np.save(os.path.join(save_path_temp, "SUV_MIP" + ".npy"), cropped_array)

        SUV_bone = np.load(image["SUV_bone"])
        cropped_array = SUV_bone[crop_from_top:crop_from_bottom, :]
        np.save(os.path.join(save_path_temp, "SUV_bone" + ".npy"), cropped_array)

        SUV_lean = np.load(image["SUV_lean"])
        cropped_array = SUV_lean[crop_from_top:crop_from_bottom, :]
        np.save(os.path.join(save_path_temp, "SUV_lean" + ".npy"), cropped_array)

        SUV_adipose = np.load(image["SUV_adipose"])
        cropped_array = SUV_adipose[crop_from_top:crop_from_bottom, :]
        np.save(os.path.join(save_path_temp, "SUV_adipose" + ".npy"), cropped_array)

        SUV_air = np.load(image["SUV_air"])
        cropped_array = SUV_air[crop_from_top:crop_from_bottom, :]
        np.save(os.path.join(save_path_temp, "SUV_air" + ".npy"), cropped_array)
        
        CT_MIP = np.load(image["CT_MIP"])
        cropped_array = CT_MIP[crop_from_top:crop_from_bottom, :]
        np.save(os.path.join(save_path_temp, "CT_MIP" + ".npy"), cropped_array)

        CT_bone = np.load(image["CT_bone"])
        cropped_array = CT_bone[crop_from_top:crop_from_bottom, :]
        np.save(os.path.join(save_path_temp, "CT_bone" + ".npy"), cropped_array)

        CT_lean = np.load(image["CT_lean"])
        cropped_array = CT_lean[crop_from_top:crop_from_bottom, :]
        np.save(os.path.join(save_path_temp, "CT_lean" + ".npy"), cropped_array)

        CT_adipose = np.load(image["CT_adipose"])
        cropped_array = CT_adipose[crop_from_top:crop_from_bottom, :]
        np.save(os.path.join(save_path_temp, "CT_adipose" + ".npy"), cropped_array)

        CT_air = np.load(image["CT_air"])
        cropped_array = CT_air[crop_from_top:crop_from_bottom, :]
        np.save(os.path.join(save_path_temp, "CT_air" + ".npy"), cropped_array)
